In [36]:
import sqlite3
import pandas as pd

In [37]:
DATA_DIR = 'data/'

In [46]:
conn = sqlite3.connect('test.db')
cur = conn.cursor()

In [39]:
cur.execute('''
                CREATE TABLE IF NOT EXISTS temperatures (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    timestamp TEXT,
                    temperature REAL
                )
            ''')
conn.commit()

In [40]:
data = pd.read_csv(DATA_DIR + 'temp-uppsala.csv', sep=";", skiprows=10)
data['DatumTid'] = data['Datum'] + ' ' + data['Tid (UTC)']

data = data[['DatumTid', 'Lufttemperatur']]

data

/var/folders/pg/6tvtzc815214nstb4nc45dcw0000gn/T/ipykernel_5484/36791995.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(DATA_DIR + 'temp-uppsala.csv', sep=";", skiprows=10)


,DatumTid,Lufttemperatur
0,1985-06-01 00:00:00,6.5
1,1985-06-01 03:00:00,5.7
2,1985-06-01 06:00:00,14.3
3,1985-06-01 09:00:00,20.5
4,1985-06-01 12:00:00,22.8
...,...,...
319228,2024-02-01 02:00:00,3.3
319229,2024-02-01 03:00:00,3.2
319230,2024-02-01 04:00:00,3.0
319231,2024-02-01 05:00:00,2.7


In [41]:
for row in data.itertuples():
    # print(row[1], row[2])
    cur.execute('''
                    INSERT INTO temperatures (timestamp, temperature)
                    VALUES (?, ?)
                ''', (row[1], row[2]))
    # f"""INSERT INTO temperatures (timestamp, temperature) VALUES ('{row[1]}', {row[2]})"""
    # elak_row1 = "'); DROP TABLE temperatures; --"
    # break

In [42]:
conn.commit()

In [55]:
# cur.execute('SELECT * FROM temperatures WHERE temperature > 30')
# user = cur.fetchone()

for row in cur.execute("SELECT MIN(temperature), MAX(temperature) FROM temperatures WHERE timestamp > '2024-02-01 00:00:00'"):
    print(row)

(2.5, 3.3)


In [44]:
# conn.close()